In [19]:
#dependencies
import pandas as pd
import numpy as np
file = "Resources/purchase_data.csv"
purc_df = pd.read_csv(file)

In [20]:
purc_df.head()
# print columns for future use
# purc_df.['Purchase ID', 'SN', 'Age', 'Gender', 'Item ID', 'Item Name', 'Price']

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


In [682]:
# find unique user nicknames for player number
player_unique = len(purc_df['SN'].unique())
print("Total number of players: " + ' {:,.0f}'.format(player_unique))

# sum all sales prices for gross revenues
pd.options.display.float_format = '{:,.2f}'.format
gross_sales = purc_df['Price'].sum()
print("Gross Sales: "+ '$ {:,.2f}'.format(gross_sales))

# number of purchases will be equal to number of rows in dataframe 
num_purc = purc_df['Purchase ID'].count()
print("Number of Purchases: " +  ' {:,.0f}'.format(num_purc))

# average purchase price
avg_price = purc_df['Price'].mean()
print("Average Price: "+ '$ {:,.2f}'.format(avg_price))

# number of unique values in items column
unique_items = len(purc_df['Item ID'].unique())
print("Number of unique items: " + ' {:,.0f}'.format(unique_items))


AttributeError: 'numpy.ndarray' object has no attribute 'head'

In [22]:
# Purchasing Analysis cont'd: present in dataframe format
Purc_Analysis = pd.DataFrame({"Gross Sales": [gross_sales],
                              "Purchase Count": [num_purc] ,
                              "Unique Item Cnt": [unique_items] ,
                              "Average Price": [avg_price], 
                              "Player Count" :[player_unique]})

# dataframe formatted in traditonal accounting style
Purc_Analysis['Gross Sales']=Purc_Analysis['Gross Sales'].map("$ {:,.0f} ".format)
Purc_Analysis['Purchase Count']=Purc_Analysis['Purchase Count']
Purc_Analysis['Unique Item Cnt']=Purc_Analysis['Unique Item Cnt']
Purc_Analysis['Average Price']=Purc_Analysis['Average Price'].map("$ {:,.2f} ".format)
Purc_Analysis['Player Count']=Purc_Analysis['Player Count']
Purc_Analysis.head()


,Gross Sales,Purchase Count,Unique Item Cnt,Average Price,Player Count
0,"$ 2,380",780,183,$ 3.05,576


In [23]:
player_unq=purc_df.drop_duplicates(['SN'], keep = 'last')
player_unq.head()
# an overview for gender demographics
gender_df=player_unq['Gender'].value_counts().reset_index()
total = gender_df['Gender'].sum()
# a new column for percentage of total players
gender_df['Perc_Gender (%)'] = gender_df['Gender']/total*100
# gender_df= gender_df.reset_index()
# gender_df=gender_df.drop(columns='level_0')
# * Percentage and Count of Male Players
gender_df.loc[gender_df['index']=='Male','Perc_Gender (%)']
gender_df.loc[gender_df['index']=='Male','Gender']
# Percentage and Count of Female Players
gender_df.loc[gender_df['index']=='Female','Perc_Gender (%)']
gender_df.loc[gender_df['index']=='Female','Gender']
# Percentage and Count of Other / Non-Disclosed
gender_df.loc[gender_df['index']=='Other / Non-Disclosed','Perc_Gender (%)']
int(gender_df.loc[gender_df['index']=='Other / Non-Disclosed','Gender'])
gender_df=gender_df.rename(columns={'index':'Gender','Gender':'Player_Count','Perc_Gender (%)':'Percentage'})
# .map("${:.2f}".format
# gender_df['Percentage']
gender_df['Percentage'] =gender_df['Percentage'].map("{:.2f} %".format)
gender_df.set_index('Gender',inplace=True)
gender_df.rename(columns={'Player_Count':'Player Count'},inplace=True)
gender_df

,Player Count,Percentage
Gender,,
Male,484,84.03 %
Female,81,14.06 %
Other / Non-Disclosed,11,1.91 %


In [97]:
### Purchasing Analysis (Gender)
## The below each broken by gender
# Purchase Count
Purchase_Count=pd.DataFrame(purc_df.groupby('Gender')['Gender'].count())
Purchase_Count.head()

# Avg Purchase Price 
Purchase_Price=pd.DataFrame(purc_df.groupby('Gender')['Price'].mean())
Purchase_Price

# Total Purchase Value
Total_Purchase = pd.DataFrame(purc_df.groupby('Gender')['Price'].sum())
Total_Purchase

# Removing duplicates required for "per person" figures
# Avg Purchase Total per Person by Gender
Avg_Purchase=pd.DataFrame(purc_df.groupby('Gender')['Price'].mean())

Purc_Analysis_Gender =pd.merge(Purchase_Count,Purchase_Price,left_index = True, right_index = True)
Purc_Analysis_Gender_2=pd.merge(Purc_Analysis_Gender,Total_Purchase, left_index = True, right_index = True, suffixes=('_Avg','_Total'))

# Normalize data by deleting all duplicates adn resortt if
remove_dup_df = purc_df.drop_duplicates(subset='SN', keep="last")
norm_df =  pd.DataFrame(remove_dup_df.groupby('Gender')['SN'].count())
Purc_Analysis_Gender_3 = pd.merge(Purc_Analysis_Gender_2, norm_df, left_index = True, right_index = True)
Purc_Analysis_Gender_3['SN']=Purc_Analysis_Gender_3['SN'].map("{:,.0f}".format)

# creating dummy columns then drop them 
Purc_Analysis_Gender_3['new_column'] = Purc_Analysis_Gender_3['Price_Total'].astype('float')
Purc_Analysis_Gender_3['new_column_2'] = Purc_Analysis_Gender_3['SN'].astype('float')
Purc_Analysis_Gender_3['Per_Person']  = Purc_Analysis_Gender_3['new_column']/Purc_Analysis_Gender_3['new_column_2']
Purc_Analysis_Gender_3d=Purc_Analysis_Gender_3
Purc_Analysis_Gender_3d=Purc_Analysis_Gender_3d.drop(columns=['new_column' , 'new_column_2'])
Purc_Analysis_Gender_3d=Purc_Analysis_Gender_3d.rename({"SN" : "Unique_Player_Count"})
Purc_Analysis_Gender_3d['Unique_Player_Count'] = Purc_Analysis_Gender_3d['SN']
Purc_Analysis_Gender_3d=Purc_Analysis_Gender_3d.drop(columns = ["SN"])

# format data frame after dropping columns
Purc_Analysis_Gender_3df=Purc_Analysis_Gender_3d
Purc_Analysis_Gender_3df['Price_Total']=Purc_Analysis_Gender_3d['Price_Total'].map("$  {:,.2f}".format)
Purc_Analysis_Gender_3df['Price_Avg']=Purc_Analysis_Gender_3d['Price_Avg'].map("$  {:,.2f}".format)
Purc_Analysis_Gender_3df['Per_Person']=Purc_Analysis_Gender_3d['Per_Person'].map("$  {:,.2f}".format)
Purc_Analysis_Gender_3df=Purc_Analysis_Gender_3df.rename(columns= {'Gender':'Purchase Count', 'Price_Avg':'Avg Purchase Price', 'Price_Total':'Total Purchase Value', 'Per_Person':'Avg Total Purchase per Person','Unique_Player_Count':'Player Count'})
Purc_Analysis_Gender_3df

,Purchase Count,Avg Purchase Price,Total Purchase Value,Avg Total Purchase per Person,Player Count
Gender,,,,,
Female,113,$ 3.20,$ 361.94,$ 4.47,81
Male,652,$ 3.02,"$ 1,967.64",$ 4.07,484
Other / Non-Disclosed,15,$ 3.35,$ 50.19,$ 4.56,11


In [562]:
# establish bins for age intervals, total of 9 bins (n)
max_age=purc_df['Age'].max()
age_bins = [0,9,14,19,24,29,34,39,max_age]
# establishing age groups as labels in string format, total of 8 groups (n-1) as expected
label_bins = ['<10', '10 - 14', '15 - 19', '20 - 24', '25 - 29', '30 - 34', '35 - 39', '40+']

# categorize existing players creating a new column "Player_Age"
# Total of 17+22+107+258+77+52+31+12 = 576 shown on the starter file thus unique player count required
agebins_df = purc_df.copy()
# agebins_df.count() returns 780 still not unique
agebins_df = agebins_df.drop_duplicates(['SN'], keep = 'last')
# agebins_df.count() returns 576 thus will be working with unique values
agebins_df["Player_Age"] = pd.cut(agebins_df["Age"], age_bins, labels = label_bins)
agebins_df=agebins_df.set_index('Player_Age')
Player_Age_gb = agebins_df.groupby('Player_Age').count()
# transform groupby object to a dataframe object for ease in calculation
Player_Age_df = pd.DataFrame(Player_Age_gb)
# ['Purchase ID', 'SN', 'Age', 'Gender', 'Item ID', 'Item Name', 'Price']
Player_Age_df['Percentage_Players'] = Player_Age_df['SN']/Player_Age_df['SN'].sum()*100
# creating a copy for formatting and dropping columns of the newly created dataframe
Age_Percentage_Only=Player_Age_df.copy()
Age_Percentage_Only=Age_Percentage_Only.drop(columns = ['Purchase ID', 'Age', 'Gender', 'Item ID', 'Item Name', 'Price'])
Age_Percentage_Only['Percentage_Players'] = Age_Percentage_Only['Percentage_Players'].map("{:,.2f} %".format)
Age_Percentage_Only.rename(columns={'SN':'Total Player Count', 'Percentage_Players':'Percentage of Players'})


,Total Player Count,Percentage of Players
Player_Age,,
<10,17,2.95 %
10 - 14,22,3.82 %
15 - 19,107,18.58 %
20 - 24,258,44.79 %
25 - 29,77,13.37 %
30 - 34,52,9.03 %
35 - 39,31,5.38 %
40+,12,2.08 %


In [592]:
age_df=purc_df.copy()
age_df["Player_Age"] = pd.cut(age_df["Age"], age_bins, labels = label_bins)
age_df=age_df.set_index('Player_Age')
# age_df.count() returns 780 since no unique values required for those belowPurchaseCount_gb = age_df.groupby('Player_Age').count()
PurchaseTotal_gb = age_df.groupby('Player_Age').sum()
PurchaseTotal_gb_df =pd.DataFrame(PurchaseTotal_gb)

# PurchaseTotal_gb_df=PurchaseTotal_gb_df['']
# avg price calculation (through groupby object>dataframe>array transformation method)
AvgPrice_gb = age_df.groupby('Player_Age').mean()
AvgPrice_gb_df = pd.DataFrame(AvgPrice_gb)
#  agebins_df.count() established above remains 576 thus consisting of only unique values
Count_Age_gb = agebins_df.groupby('Player_Age').count()
Count_Age_gb_df = pd.DataFrame(Count_Age_gb)
# unlike to the above calculations, this time code will find the count for all values...
# ...including duplicate usernames when multiple purchases made by same user
Player_Age_gb2 = age_df.groupby('Player_Age').count()
Player_Age_df = pd.DataFrame(Player_Age_gb2)
Total_Purch_df = Player_Age_df['Age']

Age_Analysis = pd.DataFrame({'Purchase Count' :Total_Purch_df,
              'Avg Purchase Price' : AvgPrice_gb_df['Price'],
              'Purchase Total' :PurchaseTotal_gb_df['Price'],
              'Count Age': Count_Age_gb_df['Age']})

Age_Analysis['Avg Purchase Per Person'] =Age_Analysis['Purchase Total']/Age_Analysis['Count Age']

# creates a dataframe with the dummy column dropped
Age_Analysis_d=Age_Analysis.drop(columns=['Count Age'])
# copy for non- formatted dataframe before calculations
Age_Analysis_df=Age_Analysis_d.copy()
Age_Analysis_df['Avg Purchase Price'] = Age_Analysis_df['Avg Purchase Price'].map("$ {:,.2f}".format)
Age_Analysis_df['Avg Purchase Per Person'] = Age_Analysis_df['Avg Purchase Per Person'].map("$ {:,.2f}".format)
Age_Analysis_df['Purchase Total'] = Age_Analysis_df['Purchase Total'].map("$ {:,.2f}".format)
Age_Analysis_df
# Age_Percentage_Only['Percentage_Players'] = Age_Percentage_Only['Percentage_Players'].map("{:,.2f} %".format)

,Purchase Count,Avg Purchase Price,Purchase Total,Avg Purchase Per Person
Player_Age,,,,
<10,23,$ 3.35,$ 77.13,$ 4.54
10 - 14,28,$ 2.96,$ 82.78,$ 3.76
15 - 19,136,$ 3.04,$ 412.89,$ 3.86
20 - 24,365,$ 3.05,"$ 1,114.06",$ 4.32
25 - 29,101,$ 2.90,$ 293.00,$ 3.81
30 - 34,73,$ 2.93,$ 214.00,$ 4.12
35 - 39,41,$ 3.60,$ 147.67,$ 4.76
40+,13,$ 2.94,$ 38.24,$ 3.19


In [621]:
user = purc_df.copy()
SN_gb = user.groupby('SN')
# Total purchase value by sum of all prices purchased by the user
total_pv =SN_gb['Price'].sum()
#  purchase count by how many prices accumulated (purhcases made) under user name 
purchase_count = SN_gb['Price'].count()
# average purchase calculated by mean of purchases/prices
avg_user = SN_gb['Price'].mean()
SN_gb_df = pd.DataFrame({'Purchase Count' :purchase_count,
              'Avg Purchase Price' : avg_user,
              'Purchase Total' :total_pv})
User_Analysis = SN_gb_df.copy()
User_Analysis=User_Analysis .sort_values(by = 'Purchase Total', ascending=False).head()
# copy dataframe containing the summary of results before formatting
User_Analysis_f=User_Analysis.copy()
User_Analysis_f['Purchase Count'] =User_Analysis_f['Purchase Count'] .map(" {:,.0f}".format)
User_Analysis_f['Avg Purchase Price'] =User_Analysis_f['Avg Purchase Price'].map("$ {:,.2f}".format)
User_Analysis_f['Purchase Total'] =User_Analysis_f['Purchase Total'] .map("$ {:,.2f}".format)
User_Analysis_f.head()

,Purchase Count,Avg Purchase Price,Purchase Total
SN,,,
Lisosia93,5,$ 3.79,$ 18.96
Idastidru52,4,$ 3.86,$ 15.45
Chamjask73,3,$ 4.61,$ 13.83
Iral74,4,$ 3.40,$ 13.62
Iskadarya95,3,$ 4.37,$ 13.10


In [662]:
#  ¨  Retrieve the Item ID, Item Name, and Item Price columns
item_df = purc_df[['Item ID', 'Item Name', 'Price']]
#  ¨ Group by Item ID and Item Name. 
item_id_name_groupby = item_df.groupby(['Item ID', 'Item Name'])
# Perform calculations to obtain purchase count, item price, and total purchase value
purchase_count = item_id_name_groupby['Price'].count()
item_price = item_id_name_groupby['Price'].mean()
total_purchase_value = item_id_name_groupby['Price'].sum()
#  ¨ Create a summary data frame to hold the results
item_data = {'Purchase Count':purchase_count,
            'Item Price' : item_price,
            'Total Purchase Value' : total_purchase_value}
item_summary_df = pd.DataFrame(data = item_data)
# Display a preview of the summary data frame
item_summary_df.head()
#  ¨ Sort the purchase count column in descending order
item_summary_df = item_summary_df.sort_values(by = 'Purchase Count', ascending=False).head() 
# Optional: give the displayed data cleaner formatting
# for this purpose, code created a copy of above dataframe in order to avoid formatting issues
# since some of the columns might be stored as strings which could lead to bigger problems in future use
item_summary_df_formatted = item_summary_df.copy()
item_summary_df_formatted['Item Price'] = item_summary_df_formatted['Item Price'].map("$ {:,.2f}".format)
item_summary_df_formatted['Total Purchase Value'] = item_summary_df_formatted['Total Purchase Value'].map("$ {:,.2f}".format)
item_summary_df_formatted

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$ 4.23,$ 50.76
145,Fiery Glass Crusader,9,$ 4.58,$ 41.22
108,"Extraction, Quickblade Of Trembling Hands",9,$ 3.53,$ 31.77
82,Nirvana,9,$ 4.90,$ 44.10
19,"Pursuit, Cudgel of Necromancy",8,$ 1.02,$ 8.16


In [672]:
# Sort the above table by total purchase value in descending order
item_summary_df2 = item_summary_df.sort_values(by = 'Total Purchase Value', ascending=False).head() 
item_summary_df2
# Optional: give the displayed data cleaner formatting
item_summary_df_formatted2 = item_summary_df2.copy()
item_summary_df_formatted2['Item Price'] = item_summary_df_formatted2['Item Price'].map("$ {:,.2f}".format)
item_summary_df_formatted2['Total Purchase Value'] = item_summary_df_formatted2['Total Purchase Value'].map("$ {:,.2f}".format)
# Display a preview of the data frame
item_summary_df_formatted2







,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$ 4.23,$ 50.76
82,Nirvana,9,$ 4.90,$ 44.10
145,Fiery Glass Crusader,9,$ 4.58,$ 41.22
108,"Extraction, Quickblade Of Trembling Hands",9,$ 3.53,$ 31.77
19,"Pursuit, Cudgel of Necromancy",8,$ 1.02,$ 8.16
